# **ติดตั้ง Dependencies**

In [ ]:
!pip install ultralytics
!pip install torch torchvision
!pip install onnx
!pip install numpy
!pip install onnxruntime
!pip install opencv-python

# **โมเดลที่มีการแก้ไขสถาปัตยกรรม (เช่น เพิ่ม DSPPF, C2fEMA, EMA) ไม่สามารถโหลดได้จากไฟล์ .pt เพียงอย่างเดียว ต้องมีไฟล์ที่ประกาศคลาสของโมดูลที่ถูกปรับแต่ง เพื่อให้ runtime สามารถ reconstruct network graph ได้ถูกต้อง**

# 1) IMPORT CUSTOM MODULES
ต้องมีการแก้ไข line 5: path ไปยังตัวของ project_root

*   sys.path.append("D:/DCAS_project")   # Windows
*   sys.path.append("/home/user/DCAS_project")  # Linux

line 7 คือ module ที่มีการเพิ่มเข้าไปใน custom yolo ที่ต้องทำการ import

In [ ]:
# =====================================================
# 1) ADD PROJECT ROOT TO PYTHON PATH
# =====================================================
import sys
sys.path.append(".")   # หรือ path เต็มของ project_root

# 2) Register Custom Layers ให้ระบบรู้จัก

Custom module ของ DCAS_EMAdp มี

*   DSPPF
*   C2fEMAdp
*   EMAdp

จึงต้องทำการ import ออกมา



In [ ]:
# =====================================================
# 2) IMPORT CUSTOM MODULES FROM ultralytics.nn
# =====================================================
from ultralytics.nn.modules import DSPPF, C2fEMA, EMA

# 3) Register custom layers ให้ runtime รู้จัก

In [ ]:
# =====================================================
# 3) REGISTER CUSTOM LAYERS INTO ULTRALYTICS
# =====================================================
import ultralytics.nn.modules as um

um.DSPPF = DSPPF
um.C2fEMA = C2fEMA
um.EMA = EMA

# 4) โหลดโมเดลที่เทรนไว้

In [ ]:
# =====================================================
# 4) LOAD MODEL
# =====================================================
from ultralytics import YOLO

model = YOLO("best.pt")   # หรือ path เต็ม

# 5) Export โมเดลเป็น ONNX

In [ ]:
# =====================================================
# 5) EXPORT TO ONNX
# =====================================================
model.export(format="onnx", imgsz=640, opset=12)

print("EXPORT DONE")

# ตัวอย่างการ import จาก yolo26 ให้เป็น onnx และแสดงผลลัพธ์

In [ ]:
from ultralytics import YOLO

# Load the YOLO26 model
model = YOLO("yolo26n.pt")

# Export the model to ONNX format
model.export(format="onnx")  # creates 'yolo26n.onnx'

# Load the exported ONNX model
onnx_model = YOLO("yolo26n.onnx")

# Run inference
results = onnx_model("https://ultralytics.com/images/bus.jpg")

annotated_image = results[0].plot()

# save img
cv2.imwrite("yolo26_result.jpg", annotated_image)

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.10.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
YOLO26n summary (fused): 122 layers, 2,408,932 parameters, 0 gradients, 5.4 GFLOPs

PyTorch: starting from 'yolo26n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 300, 6) (5.3 MB)

ONNX: starting export with onnx 1.20.1 opset 22...
ONNX: slimming with onnxslim 0.1.85...
ONNX: export success ✅ 1.8s, saved as 'yolo26n.onnx' (9.5 MB)

Export complete (2.4s)
Results saved to /content
Predict:         yolo predict task=detect model=yolo26n.onnx imgsz=640 
Validate:        yolo val task=detect model=yolo26n.onnx imgsz=640 data=/home/lq/codes/ultralytics/ultralytics/cfg/datasets/coco.yaml  
Visualize:       https://netron.app
WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading yolo26n.onnx for ONNX Runtime inference...
Using ONNX Runtime 1.24.2 with CPUExecution

True

# ตัวอย่างการ นำ weight best.pt ของ DCAS_EMAdp ที่แปลงเป็น onnx มาใช้งาน

In [ ]:
from ultralytics import YOLO
# Load the exported ONNX model
onnx_model = YOLO("/content/best.onnx")

# Run inference
results = onnx_model("/content/2024_11_25_15_03_33_exp_5_000000_NAVSTAR38USA126_flipH_blur.png")
# save img
annotated_image = results[0].plot()
cv2.imwrite("DCAS_EMAdp_result.jpg", annotated_image)

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading /content/best.onnx for ONNX Runtime inference...
Using ONNX Runtime 1.24.2 with CPUExecutionProvider

image 1/1 /content/2024_11_25_15_03_33_exp_5_000000_NAVSTAR38USA126_flipH_blur.png: 640x640 1 object, 475.4ms
Speed: 4.3ms preprocess, 475.4ms inference, 3.4ms postprocess per image at shape (1, 3, 640, 640)


True